In [1]:
import re
import pandas as pd

In [ ]:
file = open("./WS-Chat.txt", "r", encoding="utf-8")
data = file.read()
# print(data)

In [3]:
# Re expression to get only msg, and remove time and date
# i.e: "26/12/2000, 12:00 - "
pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'
# pattern='\d{1,2}\/\d{1,2}\/\d{2},\s\d{1,2}:\d{2}\s(?:AM|PM)'  #This Pattern fo rAM/PM timing

In [ ]:
message = re.split(pattern, data, flags=re.I)[1:]
print(message)

In [ ]:
dates = re.findall(pattern, data)
print(dates)

In [ ]:
df = pd.DataFrame({'user_message': message, 'message_date': dates})

# Converting message_date type
df['message_date'] = pd.to_datetime(
    df['message_date'], format='%d/%m/%Y, %H:%M - ')

df.rename(columns={"message_date": "date"}, inplace=True)
df.head()

In [7]:
df.shape

(3290, 2)

In [ ]:
user = []
messages = []
for message in df["user_message"]:
    entry = re.split("([\w\W]+?):\s", message)
    if entry[1:]:
        user.append(entry[1])
        messages.append(entry[2])
    else:
        user.append("group_notification")
        messages.append(entry[0])

df["user"] = user
df["message"] = messages
df.drop(columns=["user_message"], inplace=True)

df.head()

In [ ]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month_name()
df["day"] = df["date"].dt.day
df["hour"] = df["date"].dt.hour
df["minute"] = df["date"].dt.minute
df.head()

In [10]:
words = []
for message in df["message"]:
    words.extend(message.split())
len(words)

14651

In [ ]:
# Get number of media
df[df["message"] == "<Media omitted>\n"]

In [ ]:
# Get number of link

from urlextract import URLExtract
extractor = URLExtract()
urls = extractor.find_urls(
    "This consists of link www.google.com and www.facebook.com")
print(len(urls))
print(urls)

In [ ]:
# The most buesiest user for group only
x = df["user"].value_counts().head()
x

In [ ]:
import matplotlib.pyplot as plt
name = x.index
count = x.values

plt.bar(name, count)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# round((df["user"].value_counts() / df.shape[0])*100, 2)
# round((df["user"].value_counts() / df.shape[0])*100, 2).reset_index()
round((df["user"].value_counts() / df.shape[0])*100,
      2).reset_index().rename(columns={"index": "name", "user": "percentage"})

In [ ]:
df["message"].str.cat(sep="")

In [17]:
# Remvoing Group notification
temp = df[df["user"] != "group_notification"]

# Removing Media omitted
temp = temp[temp["message"] != "<Media omitted>\n"]

# Removing stop words
# Here chat could be in hindi, so we can't use default stop words removal from nltk
# We create a file of common Hinglish words and use it

stopWords_file = open("./stop_hinglish.txt", "r")
stop_words = stopWords_file.read()
# print(stop_words)

In [18]:
words = []

for message in temp["message"]:
    for word in message.lower().split():
        if (word not in stop_words):
            words.append(word)

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
# Finding most cmn emojis
import emoji

emojis = []
for message in df["message"]:
    emojis.extend([c for c in message if c in emoji.EMOJI_DATA])

pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis))))

In [21]:
# Timeline analysis
df["month_num"] = df["date"].dt.month

In [ ]:
timeline = df.groupby(["year", "month_num", "month"]).count()[
    "message"].reset_index()
timeline

In [ ]:
time = []
for i in range(timeline.shape[0]):
    time.append(timeline["month"][i] + "-" + str(timeline["year"][i]))
timeline["time"] = time
timeline

In [ ]:
plt.plot(timeline["time"], timeline["message"])
plt.xticks(rotation=90)
plt.show()

In [25]:
# Daily timeline analysis
df["separate_date"] = df["date"].dt.date

daily_timeline = df.groupby("separate_date").count()["message"].reset_index()

In [ ]:
plt.figure(figsize=(18, 10))
plt.plot(daily_timeline["separate_date"], daily_timeline["message"])

In [ ]:
df["day_name"] = df["date"].dt.day_name()
df["day_name"].value_counts()

In [ ]:
df["hour"]

In [38]:
# Heat map for time
# Creating peiod clm from HR, i.e: Hr "16" period: 16-17

period = []
for hour in df[["day_name", "hour"]]["hour"]:
    if hour == 23:
        period.append(str(hour)+"-"+str("00"))
    elif hour == 0:
        period.append(str("00")+"-"+str(hour+1))
    else:
        period.append(str(hour)+"-"+str(hour+1))

df["period"] = period

In [ ]:
df.sample(5)

In [ ]:
import seaborn as sns

plt.figure(figsize=(20, 6))
sns.heatmap(df.pivot_table(index="day_name", columns="period", values="message", aggfunc="count").fillna(
    0))  # There are time of the daty with out any msg so we fill it with 0
plt.yticks(rotation="horizontal")
plt.show()